In [ ]:
# Imports
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Sweep configuration
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'img_size': {'values': [128]},  # Set a reasonably large size for MobileNetV2
        'epochs': {'values': [10]},
        'experiment': {'values': ['transfer_learning_with_differential_lr']}
    }
}

In [ ]:
# Initialize W&B sweep
sweep_id = wandb.sweep(sweep_config, project="5-flowers-transfer-learning-with-differential-lr")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: zv76y4sm
Sweep URL: https://wandb.ai/pritkudale-vizuara/5-flowers-transfer-learning-with-differential-lr/sweeps/zv76y4sm


In [ ]:
# Define an exponentially decaying learning rate schedule


In [ ]:
# Train function
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        # Helper functions
        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Prepare datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )


        # Build model
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
            include_top=False,    # Remove ImageNet head
            weights="imagenet"    # Use pretrained weights
        )
        base_model.trainable = True  # Freeze base

        model = keras.Sequential([
            base_model,
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.BatchNormalization(),       # Add BatchNorm
            keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
        ])

        # Different learning rates for base and head
        base_learning_rate = 1e-5
        head_learning_rate = config.learning_rate  # Example: 1e-4

        optimizer = tf.keras.optimizers.Adam()

        for layer in model.layers[0].layers:  # Layers inside base_model
            layer.trainable = True
            layer._learning_rate = base_learning_rate

        for layer in model.layers[1:]:
            layer._learning_rate = head_learning_rate

        # Compile the model with the learning rate schedule
        model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=["accuracy"])

        # Visualize model
        model.summary()

        keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file="model_visualization.png")

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

In [ ]:
# Launch W&B agent
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: nyc9rgwu with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: transfer_learning_with_differential_lr
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: vizuara-info (pritkudale-vizuara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,269,509 (8.66 MB)

...

Epoch 1/10
    413/Unknown 305s 654ms/step - accuracy: 0.5937 - loss: 1.6272

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 331s 716ms/step - accuracy: 0.5938 - loss: 1.6265 - val_accuracy: 0.2432 - val_loss: 11.1605
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 313s 696ms/step - accuracy: 0.7292 - loss: 0.7972 - val_accuracy: 0.3027 - val_loss: 7.8106
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 380s 839ms/step - accuracy: 0.7564 - loss: 0.7181 - val_accuracy: 0.3459 - val_loss: 8.2088
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 382s 839ms/step - accuracy: 0.7950 - loss: 0.5889 - val_accuracy: 0.3135 - val_loss: 8.3817
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 321s 691ms/step - accuracy: 0.8183 - loss: 0.5326 - val_accuracy: 0.6243 - val_loss: 2.3817
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 322s 778ms/step - accuracy: 0.8408 - loss: 0.4601 - val_accuracy: 0.5162 - val_loss: 4.5308
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 292s 706ms/step - accuracy: 0.8288 - loss: 0.5077 - val_accuracy: 0.4838 - val_loss: 3.5529
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 326s 717ms/step - accuracy: 0.8578 - loss: 0.4

batch/accuracy,▁▁▁▂▂▄▅▅▅▅▅▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▅▇▇▇▇▇█▇▇▇▇██▇
batch/batch_step,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▇▇▆▆▆▆▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▄▅▆▆▇▇███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▄▃▂▂▂▁▁▁
epoch/val_accuracy,▁▂▃▂▇▅▅▆█▅
epoch/val_loss,█▅▆▆▁▃▂▂▁▃
batch/accuracy,0.87074


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
